# Lab | Comparing regression models


For this lab, we will be using the same dataset we used in the previous labs. Load the cleaned categorical and numerical dataframes that you saved at the end of Monday's labs.



## Instructions

Concatenate Numerical and Categorical dataframes into one dataframe called data. Split into X=features y=target (total_claim_amount).

1. In this final lab, we will model our data. Import sklearn train_test_split and separate the data.

2. Separate X_train and X_test into numerical and categorical (X_train_cat , X_train_num , X_test_cat , X_test_num)

3. Use X_train_num to fit scalers. Transform BOTH X_train_num and X_test_num.

4. Encode the categorical variables X_train_cat and X_test_cat

5. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

6. Try a simple linear regression with all the data to see whether we are getting good results.

7. Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

8. Use the function to check LinearRegressor and KNeighborsRegressor.

9. You can check also the MLPRegressor for this task!

10. Check and discuss the results.



In [22]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

In [23]:
categorical = pd.read_csv('categorical.csv')
numerical = pd.read_csv('numerical.csv')

data = pd.concat([numerical, categorical], axis=1)
pd.set_option('display.max_columns',50)
data


,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,2763.519279,56274,69,32,5,0,1,384.811147,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate Auto,L3,Offer1,Agent,Two-Door Car,Medsize
1,12887.431650,48767,108,18,38,0,2,566.472247,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal Auto,L3,Offer1,Agent,Two-Door Car,Medsize
2,7645.861827,0,106,18,65,0,7,529.881344,California,No,Basic,Bachelor,Other,M,Suburban,Married,Corporate Auto,L2,Offer1,Call Center,SUV,Medsize
3,2813.692575,43836,73,12,44,0,1,138.130879,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal Auto,L1,Offer1,Agent,Four-Door Car,Medsize
4,8256.297800,62902,69,14,94,0,2,159.383042,Oregon,Yes,Basic,Bachelor,Employed,F,Rural,Married,Personal Auto,L3,Offer2,Web,Two-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7784,4100.398533,47761,104,16,58,0,1,541.282007,California,No,Premium,College,Employed,F,Suburban,Single,Personal Auto,L2,Offer1,Branch,Four-Door Car,Large
7785,3096.511217,21604,79,14,28,0,1,379.200000,California,Yes,Extended,College,Employed,F,Suburban,Divorced,Corporate Auto,L3,Offer1,Branch,Four-Door Car,Medsize
7786,8163.890428,0,85,9,37,3,2,790.784983,California,No,Extended,Bachelor,Other,M,Suburban,Single,Corporate Auto,L2,Offer1,Branch,Four-Door Car,Medsize
7787,7524.442436,21941,96,34,3,0,3,691.200000,California,No,Extended,College,Employed,M,Suburban,Married,Personal Auto,L2,Offer3,Branch,Four-Door Car,Large


In [24]:
X = data.drop('total_claim_amount',axis=1)
y = data['total_claim_amount']

X['employmentstatus'].unique()

array(['Employed', 'Other'], dtype=object)

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=127)

In [26]:
X_train_cat = X_train.select_dtypes(include='object')
X_train_num = X_train.select_dtypes(include=np.number)
X_test_cat = X_test.select_dtypes(include='object')
X_test_num = X_test.select_dtypes(include=np.number)

In [27]:
scaler = StandardScaler().fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

In [28]:
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns)
X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns)

X_train_num_scaled

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,-1.070776,0.580003,-0.780736,0.492344,-1.719259,-0.421768,-0.830449
1,0.046558,-0.335269,-0.138256,-1.204886,-0.141848,-0.421768,1.597197
2,1.473817,0.480658,2.202208,-0.905375,1.041211,2.814729,0.383374
3,-0.355550,-0.323310,-1.010194,0.192832,1.399713,-0.421768,0.383374
4,-0.616993,-0.886651,1.422053,-0.805538,-0.070147,-0.421768,-0.830449
...,...,...,...,...,...,...,...
6226,-0.764900,-0.156256,0.412441,-0.206516,1.758216,-0.421768,-0.830449
6227,-0.404765,-1.273861,-1.056085,-0.705701,-0.751302,-0.421768,2.406412
6228,1.514005,-0.244868,0.274767,0.492344,-1.038104,-0.421768,-0.425841
6229,0.243312,-1.273861,0.412441,-0.206516,-1.002254,-0.421768,0.787981


In [29]:
ordinal_cols = ['coverage', 'employmentstatus', 'location_code', 'vehicle_size', 'education']
one_hot_cols = list(X_train_cat.drop(ordinal_cols, axis=1).columns)

for col in ordinal_cols:
    print(X_train_cat[col].unique())

['Basic' 'Extended' 'Premium']
['Employed' 'Other']
['Suburban' 'Rural' 'Urban']
['Medsize' 'Small' 'Large']
['High School or Below' 'Bachelor' 'Postgraduate' 'College']


In [30]:
ordinal_mapping = {
    'coverage': {"Basic": 0, "Extended": 1, "Premium": 2},
    'employmentstatus': {"Employed": 0, "Other": 1},
    'location_code': {"Rural": 0, "Suburban": 1, "Urban": 2},
    'vehicle_size': {"Small": 0, "Medsize": 1, "Large": 2},
    'education': {"High School or Below": 0, "College": 1, "Bachelor": 2, "Postgraduate": 3}
}

In [31]:
for col, mapping in ordinal_mapping.items():
    X_train_cat[col] = X_train_cat[col].map(mapping)
    X_test_cat[col] = X_test_cat[col].map(mapping)


In [32]:
X_train_cat[ordinal_mapping.keys()]

,coverage,employmentstatus,location_code,vehicle_size,education
7416,0,0,1,1,0
872,1,1,1,1,2
1957,1,0,0,1,3
3546,0,0,2,1,2
1284,0,1,1,1,1
...,...,...,...,...,...
3108,1,0,1,1,3
1116,0,1,2,1,3
866,1,0,1,0,0
60,1,1,1,2,2


In [33]:
X_train_cat_encoded1 = X_train_cat[ordinal_mapping.keys()].reset_index(drop=True)
X_train_cat_encoded1

,coverage,employmentstatus,location_code,vehicle_size,education
0,0,0,1,1,0
1,1,1,1,1,2
2,1,0,0,1,3
3,0,0,2,1,2
4,0,1,1,1,1
...,...,...,...,...,...
6226,1,0,1,1,3
6227,0,1,2,1,3
6228,1,0,1,0,0
6229,1,1,1,2,2


In [34]:
X_test_cat_encoded1 = X_test_cat[ordinal_mapping.keys()].reset_index(drop=True)
X_test_cat_encoded1

,coverage,employmentstatus,location_code,vehicle_size,education
0,0,1,1,1,0
1,1,0,2,1,1
2,2,0,1,1,0
3,1,0,1,2,2
4,1,1,1,1,1
...,...,...,...,...,...
1553,0,0,1,0,3
1554,0,1,1,2,1
1555,1,0,0,1,1
1556,0,0,0,0,2


In [35]:
onehot = OneHotEncoder(handle_unknown='error', drop='first').fit(X_train_cat[one_hot_cols])

encoded_columns = onehot.get_feature_names_out(one_hot_cols)
encoded_train = onehot.transform(X_train_cat[one_hot_cols])
encoded_test = onehot.transform(X_test_cat[one_hot_cols])

X_train_cat_encoded2 = pd.DataFrame(encoded_train.toarray(), columns=encoded_columns)
X_train_cat_encoded2

,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_L2,policy_L3,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6226,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6227,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6228,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6229,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [36]:
X_test_cat_encoded2 = pd.DataFrame(encoded_test.toarray(), columns=encoded_columns)
X_test_cat_encoded2

,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_L2,policy_L3,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1554,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1555,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1556,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [37]:
X_train = pd.concat((X_train_cat_encoded1,X_train_cat_encoded2, X_train_num_scaled),axis=1)

In [38]:
X_train

,coverage,employmentstatus,location_code,vehicle_size,education,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_L2,policy_L3,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.070776,0.580003,-0.780736,0.492344,-1.719259,-0.421768,-0.830449
1,1,1,1,1,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046558,-0.335269,-0.138256,-1.204886,-0.141848,-0.421768,1.597197
2,1,0,0,1,3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.473817,0.480658,2.202208,-0.905375,1.041211,2.814729,0.383374
3,0,0,2,1,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.355550,-0.323310,-1.010194,0.192832,1.399713,-0.421768,0.383374
4,0,1,1,1,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,-0.616993,-0.886651,1.422053,-0.805538,-0.070147,-0.421768,-0.830449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6226,1,0,1,1,3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.764900,-0.156256,0.412441,-0.206516,1.758216,-0.421768,-0.830449
6227,0,1,2,1,3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.404765,-1.273861,-1.056085,-0.705701,-0.751302,-0.421768,2.406412
6228,1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.514005,-0.244868,0.274767,0.492344,-1.038104,-0.421768,-0.425841
6229,1,1,1,2,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.243312,-1.273861,0.412441,-0.206516,-1.002254,-0.421768,0.787981


In [39]:
X_test = pd.concat((X_test_cat_encoded1,X_test_cat_encoded2, X_test_num_scaled),axis=1)
X_test

,coverage,employmentstatus,location_code,vehicle_size,education,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_L2,policy_L3,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0,1,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.216701,-1.273861,-1.010194,1.191203,-1.683408,-0.421768,-0.425841
1,1,0,2,1,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.486422,0.525246,0.182984,0.492344,-0.249398,-0.421768,-0.425841
2,2,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.616552,0.577783,0.825464,1.790225,-1.540007,-0.421768,-0.830449
3,1,0,1,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.140085,-0.077747,1.926859,-1.005212,-0.464500,1.735897,2.406412
4,1,1,1,1,1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.838329,-1.273861,1.651510,-0.705701,-0.572051,-0.421768,1.192589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,0,0,1,0,3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.005129,1.259417,-0.551279,-1.105049,1.112911,1.735897,-0.830449
1554,0,1,1,2,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.475474,-0.526174,-1.056085,1.390877,0.109104,-0.421768,2.406412
1555,1,0,0,1,1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.122842,0.337322,-0.184148,0.092995,-0.679601,-0.421768,-0.021234
1556,0,0,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.025124,1.395764,-0.688954,-1.105049,-1.038104,-0.421768,-0.830449


In [40]:
lr = linear_model.LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [41]:
pred = lr.predict(X_train)
r2_score(y_train, pred)

0.523758920953129

In [42]:
pred2 = lr.predict(X_test)
r2_score(y_test, pred2)

0.5103604665292767

In [43]:
def train_and_test(models, X_train, y_train, X_test, y_test):
    results = {}
    
    for model in models:
        model.fit(X_train, y_train)
        
        predictions = model.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        
        results[model_name] = mse,r2
    
    return results

In [45]:
models = [ LinearRegression(), KNeighborsRegressor(), MLPRegressor()]

def train_and_test(models, X_train, y_train, X_test, y_test):
    
    results = {}
    
    for model in models:
        model.fit(X_train, y_train)
        
        predictions = model.predict(X_test)
        r2 = r2_score(y_test, predictions)
        
        results[model] = (r2)
    
    return results

results = train_and_test(models, X_train, y_train, X_test, y_test)

/Users/iremnisa.kilinc/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [46]:
results

{LinearRegression(): 0.5103604665292767,
 KNeighborsRegressor(): 0.5661307584038349,
 MLPRegressor(): 0.7475224959970941}